# Loading Libraries and Data

In [64]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)

In [69]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.feature_selection import f_classif, SelectKBest

In [2]:
loans_df = pd.read_csv('data/loan.csv')

In [3]:
loans_df.head()

,loan_id,account_id,date,amount,duration,payments,status,account_id.1,district_id,frequency,...,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,4959,2,1994-01-05,80952,24,3373.0,A,2,1,POPLATEK MESICNE,...,0,1,1,100.0,12541,0.2,0.43,167,85677.0,99107
1,4961,19,1996-04-29,30276,12,2523.0,B,19,21,POPLATEK MESICNE,...,7,1,7,67.0,9104,1.5,2.07,123,2299.0,2354
2,4962,25,1997-12-08,30276,12,2523.0,A,25,68,POPLATEK MESICNE,...,18,2,6,57.2,9893,4.0,4.72,96,5623.0,5887
3,4967,37,1998-10-14,318480,60,5308.0,D,37,20,POPLATEK MESICNE,...,3,1,4,58.4,8547,2.6,3.64,120,1563.0,1542
4,4968,38,1998-04-19,110736,48,2307.0,C,38,19,POPLATEK TYDNE,...,3,1,4,52.7,8402,3.1,3.98,120,999.0,1099


# Exploring the Data

In [14]:
loans_df.shape

(682, 27)

In [15]:
loans_df.isnull().sum()

loan_id          0
account_id       0
date             0
amount           0
duration         0
payments         0
status           0
account_id.1     0
district_id      0
frequency        0
date.1           0
district_id.1    0
A2               0
A3               0
A4               0
A5               0
A6               0
A7               0
A8               0
A9               0
A10              0
A11              0
A12              8
A13              0
A14              0
A15              8
A16              0
dtype: int64

In [18]:
loans_df.describe()

,loan_id,account_id,amount,duration,payments,district_id,population,num_muni_499,num_muni_1999,num_muni_9999,num_muni_10K,num_cities,urban_ratio,ave_salary,95_unemployment_rate,96_unemployment_rate,num_entrepreneurs_per1000,95_num_crimes,96_num_crimes
count,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,6.820000e+02,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,674.000000,682.000000,682.000000,674.000000,682.000000
mean,6172.466276,5824.162757,151410.175953,36.492669,4190.664223,37.489736,2.720522e+05,39.673021,21.557185,5.648094,1.712610,5.406158,68.212903,9502.986804,2.842136,3.528431,121.218475,14861.621662,16390.818182
std,682.579279,3283.512681,113372.406310,17.075219,2215.830344,25.184326,3.583320e+05,33.673348,15.780258,4.588747,1.110404,2.868251,20.245469,1323.150982,1.881095,2.146775,23.366091,27072.441726,31299.743394
min,4959.000000,2.000000,4980.000000,12.000000,304.000000,1.000000,4.282100e+04,0.000000,0.000000,0.000000,0.000000,1.000000,33.900000,8110.000000,0.200000,0.430000,81.000000,818.000000,888.000000
25%,5577.500000,2967.000000,66732.000000,24.000000,2477.000000,13.000000,9.208400e+04,5.000000,10.000000,2.000000,1.000000,4.000000,52.000000,8544.000000,1.500000,1.960000,106.000000,2166.000000,2305.000000
50%,6176.500000,5738.500000,116928.000000,36.000000,3934.000000,39.000000,1.246050e+05,35.000000,23.000000,5.000000,1.000000,6.000000,62.100000,8991.000000,2.700000,3.490000,116.000000,3732.500000,3868.000000
75%,6752.500000,8686.000000,210654.000000,48.000000,5813.500000,60.000000,2.261220e+05,65.000000,33.000000,8.000000,2.000000,7.000000,87.700000,9897.000000,3.950000,4.790000,132.000000,6949.000000,6872.000000
max,7308.000000,11362.000000,590820.000000,60.000000,9910.000000,77.000000,1.204953e+06,151.000000,70.000000,20.000000,5.000000,11.000000,100.000000,12541.000000,7.300000,9.400000,167.000000,85677.000000,99107.000000


In [17]:
loans_df.head()

,loan_id,account_id,date,amount,duration,payments,status,account_id.1,district_id,frequency,...,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,4959,2,1994-01-05,80952,24,3373.0,A,2,1,POPLATEK MESICNE,...,0,1,1,100.0,12541,0.2,0.43,167,85677.0,99107
1,4961,19,1996-04-29,30276,12,2523.0,B,19,21,POPLATEK MESICNE,...,7,1,7,67.0,9104,1.5,2.07,123,2299.0,2354
2,4962,25,1997-12-08,30276,12,2523.0,A,25,68,POPLATEK MESICNE,...,18,2,6,57.2,9893,4.0,4.72,96,5623.0,5887
3,4967,37,1998-10-14,318480,60,5308.0,D,37,20,POPLATEK MESICNE,...,3,1,4,58.4,8547,2.6,3.64,120,1563.0,1542
4,4968,38,1998-04-19,110736,48,2307.0,C,38,19,POPLATEK TYDNE,...,3,1,4,52.7,8402,3.1,3.98,120,999.0,1099


In [20]:
loans_df.groupby(['status']).agg({'account_id':'count', 'amount':'mean', 'payments':'mean', 'duration':'mean'})

,account_id,amount,payments,duration
status,,,,
A,203,91641.458128,4264.137931,22.226601
B,31,140720.903226,5396.258065,25.548387
C,403,171410.352357,3938.535980,43.444169
D,45,249284.533333,5286.644444,46.133333


In [21]:
loans_df.groupby(['duration', 'status']).agg({'account_id':'count', 'amount':'mean', 'payments':'mean'})

account_id         amount     payments
duration status                                        
12       A               93   55230.580645  4602.548387
         B               10   58425.600000  4868.800000
         C               27   47012.888889  3917.740741
         D                1   36204.000000  3017.000000
24       A               64   93229.500000  3884.562500
         B               11  136546.909091  5689.454545
         C               57   98216.421053  4092.350877
         D                6  104172.000000  4340.500000
36       A               32  159640.875000  4434.468750
         B                7  216601.714286  6016.714286
         C               83  128038.120482  3556.614458
         D                8  184297.500000  5119.375000
48       A               11  157274.181818  3276.545455
         B                2  147672.000000  3076.500000
         C              111  201648.864865  4201.018018
         D               14  283100.571429  5897.928571
60       A                3  220520.000000  3675.333333
         B                1  464520.000000  7742.000000
         C              125  233604.000000  3893.400000
         D               16  319923.750000  5332.062500

# Transforming the Data

In [ ]:
loans_df.drop(columns=['account_id.1', 'district_id.1']

In [16]:
loans_df.rename(columns = {'date':'loan_date', 'date.1':'acct_date', 'A2':'dist_name', 'A3':'region', 'A4':'population', 'A5':'num_muni_499', 'A6':'num_muni_1999', 'A7':'num_muni_9999', 'A8':'num_muni_10K', 'A9':'num_cities', 'A10':'urban_ratio', 'A11':'ave_salary', 'A12':'95_unemployment_rate', 'A13':'96_unemployment_rate', 'A14':'num_entrepreneurs_per1000', 'A15':'95_num_crimes', 'A16':'96_num_crimes'}, inplace=True)

In [20]:
loans_df.columns

Index(['loan_id', 'account_id', 'loan_date', 'amount', 'duration', 'payments',
       'status', 'district_id', 'frequency', 'acct_date', 'dist_name',
       'region', 'population', 'num_muni_499', 'num_muni_1999',
       'num_muni_9999', 'num_muni_10K', 'num_cities', 'urban_ratio',
       'ave_salary', '95_unemployment_rate', '96_unemployment_rate',
       'num_entrepreneurs_per1000', '95_num_crimes', '96_num_crimes'],
      dtype='object')

In [17]:
loans_df.nunique()

loan_id                      682
account_id                   682
loan_date                    559
amount                       645
duration                       5
payments                     577
status                         4
district_id                   77
frequency                      3
acct_date                    557
dist_name                     77
region                         8
population                    77
num_muni_499                  53
num_muni_1999                 36
num_muni_9999                 17
num_muni_10K                   6
num_cities                    11
urban_ratio                   70
ave_salary                    76
95_unemployment_rate          41
96_unemployment_rate          73
num_entrepreneurs_per1000     44
95_num_crimes                 75
96_num_crimes                 76
dtype: int64

In [31]:
loans_df['status'] = np.where((loans_df.status == 'A') | (loans_df.status == 'C'), True, False )

In [34]:
loans_df['loan_date'] = pd.to_datetime(loans_df['loan_date'])
loans_df['acct_date'] = pd.to_datetime(loans_df['acct_date'])

In [56]:
numerical_variables = ['loan_date', 'amount', 'payments', 'acct_date', 'population', 'num_muni_499', 
                     'num_muni_1999', 'num_muni_9999', 'num_muni_10K', 'num_cities', 'urban_ratio', 
                     'ave_salary', '95_unemployment_rate', '96_unemployment_rate', 'num_entrepreneurs_per1000', 
                     '95_num_crimes', '96_num_crimes']
nominal_variables = ['duration', 'dist_name', 'region']
ordinal_variable = ['frequency']

In [57]:
#Using ColumnTransformer
variable_transformer = ColumnTransformer(transformers=[('continuous', 'passthrough', numerical_variables),
                                                      ('ordinal', OrdinalEncoder(), ordinal_variable),
                                                      ('nominal', OneHotEncoder(sparse=False), nominal_variables)],
                                                      remainder='drop')

In [58]:
#Fit the above transformer
variable_transformer.fit(loans_df)

ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
                  transformer_weights=None,
                  transformers=[('continuous', 'passthrough',
                                 ['loan_date', 'amount', 'payments',
                                  'acct_date', 'population', 'num_muni_499',
                                  'num_muni_1999', 'num_muni_9999',
                                  'num_muni_10K', 'num_cities', 'urban_ratio',
                                  'ave_salary', '95_unemployment_rate',
                                  '96_unemployment_rate',
                                  'num_entrepreneurs_per1000', '95_num_crimes',
                                  '96_num_crimes']),
                                ('ordinal',
                                 OrdinalEncoder(categories='auto',
                                                dtype=<class 'numpy.float64'>),
                                 ['frequency']),
                           

In [59]:
#Transform the loans dataframe
loans_transformed = variable_transformer.transform(loans_df)

In [60]:
#Nominal column headings
nominal_names = variable_transformer.named_transformers_['nominal'].categories_

transformed_nominal_columns = []

for column, name_list in zip(nominal_variables, nominal_names):
    for name in name_list:
        transformed_nominal_columns.append('_'.join([column, str(name)]))

In [61]:
#Column headings
transformed_columns = numerical_variables + ordinal_variable + transformed_nominal_columns

In [62]:
#Converting back to dataframe
X_dataset = pd.DataFrame(loans_transformed, columns=transformed_columns)
X_dataset.head()

,loan_date,amount,payments,acct_date,population,num_muni_499,num_muni_1999,num_muni_9999,num_muni_10K,num_cities,...,dist_name_Zlin,dist_name_Znojmo,region_Prague,region_central Bohemia,region_east Bohemia,region_north Bohemia,region_north Moravia,region_south Bohemia,region_south Moravia,region_west Bohemia
0,1994-01-05,80952,3373,1993-02-26,1204953,0,0,0,1,1,...,0,0,1,0,0,0,0,0,0,0
1,1996-04-29,30276,2523,1995-04-07,103347,87,16,7,1,7,...,0,0,0,0,0,0,0,1,0,0
2,1997-12-08,30276,2523,1996-07-28,228848,15,40,18,2,6,...,0,0,0,0,0,0,1,0,0,0
3,1998-10-14,318480,5308,1997-08-18,70646,94,14,3,1,4,...,0,0,0,0,0,0,0,1,0,0
4,1998-04-19,110736,2307,1997-08-08,51428,50,11,3,1,4,...,0,0,0,0,0,0,0,1,0,0


# Feature Selection

In [72]:
#Set variables
X = X_dataset.drop(columns=['loan_date', 'acct_date'])
y = loans_df['status']

In [73]:
#Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [74]:
#Scaling the data
loan_scaler = StandardScaler()
X_train_scale = loan_scaler.fit_transform(X_train)
X_test_scale = loan_scaler.fit_transform(X_test)

In [75]:
#use SelectKBest
loan_selector_anova = SelectKBest(score_func = f_classif, k = 3)
loan_selector_anova.fit(X_train_scale, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').